In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
#from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
df = pd.read_csv('/content/AAPL_short_volume.csv')


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.info()

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.sort_index(ascending=True, inplace=True)
df

In [ ]:
!pip

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler_values = scaler.fit_transform(df[df.columns])

In [ ]:
scaler_values

In [ ]:
df_scaled = pd.DataFrame(scaler_values, columns=df.columns, index=df.index)
df_scaled

In [ ]:
plt.rcParams['figure.figsize'] = [15, 7]
figure, axes = plt.subplots(6)

for ax, col in zip(axes, df_scaled.columns):
  ax.plot(df_scaled[col])
  ax.set_title(col)
  ax.axes.xaxis.set_visible(False)



In [ ]:
import numpy as np
def create_sequence(data, window_size):
  X = []
  Y = []

  for i in range(window_size, len(data)):
    X.append(data[i-window_size:i, :])
    Y.append(data[i, :])

  return np.array(X), np.array(Y)


In [ ]:
window_size = 60
X, y = create_sequence(df_scaled.values, window_size)

In [ ]:
X.shape, y.shape

In [ ]:
X

In [ ]:
y

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

In [ ]:
model = keras.Sequential([
    keras.layers.LSTM(units = 50, return_sequences= True, input_shape = (X_train.shape[1], X_train.shape[2])),
    keras.layers.Dropout(0.3),

    keras.layers.LSTM(units = 50, return_sequences= True),
    keras.layers.Dropout(0.3),

    keras.layers.LSTM(units = 50, return_sequences= False),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(y_train.shape[1])
])

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'mean_squared_error',
              metrics = ['RootMeanSquaredError'])



In [ ]:
es = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)


In [ ]:
#history = model.fit(X_train, y_train, validation_split = 0.2, epochs = 100, batch_size=32, callbacks = [es])

lstm_model = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[es],
    verbose=1
)


In [ ]:
lstm_model.history

In [ ]:
predictions = model.predict(X_test)

In [ ]:
# FORECASTING THE DATA + INVERESE SCALING IN ORDER TO MATCH
predictions = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)




In [ ]:
predictions[:10]

In [ ]:
plt.figure(figsize=(16, 8))

for i, col in enumerate(df_scaled.columns):
    plt.subplot(2, 3, i + 1)
    plt.plot(y_test_rescaled[:, i], color='blue', label=f'Actual {col}')
    plt.plot(predictions[:, i], color='red', label=f'Predicted {col}')
    plt.title(f'{col} Prediction')
    plt.xlabel('Time')
    plt.ylabel(f'{col} Price')
    plt.legend()


